In [1]:
import os
import re
import asyncio
import nest_asyncio
from aiogram import Bot, Dispatcher, types, F
from aiogram.enums import ParseMode
from aiogram.client.default import DefaultBotProperties
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram.filters import Command
from aiogram.types import FSInputFile, InlineKeyboardMarkup, InlineKeyboardButton, CallbackQuery
from openpyxl import load_workbook
from moviepy.editor import VideoFileClip
import whisper
import difflib

nest_asyncio.apply()

TOKEN = os.getenv("TELEGRAM_BOT_TOKEN", "7481970874:AAGtUTbF11D46zz6AtCndzZBN_rJR2TnPXg")

VIDEO_PATH = "Extra1.mp4"
FRAGMENTS_DIR = "fragments"
USER_AUDIO_DIR = "user_audio"
EXCEL_PATH = "blocks.xlsx"

os.makedirs(FRAGMENTS_DIR, exist_ok=True)
os.makedirs(USER_AUDIO_DIR, exist_ok=True)

keywords = ['horno', 'español', 'quieres', 'llamo', 'perro', 'museo', 'americano', 'coches', 'chicas', 'vivo', 'correo',
            'dormitorio', 'ducha', 'estupendo', 'estás', 'quedar', 'tienes', 'cama', 'carta', 'chica', 'chico',
            'compras', 'dije', 'duerme', 'factura', 'gracias', 'hablar', 'queda', 'rápido', 'tomar', 'abajo', 'acabó',
            'amigos', 'años', 'baño', 'bici', 'bicicleta', 'celebrarlo', 'corresponsal', 'dices', 'digas', 'dormir',
            'echo', 'favor', 'febrero', 'fuertes', 'gusta', 'hablo', 'hombres', 'importa']

bigrams = ['el horno', 'en el', 'el perro', 'me llamo', 'un museo', 'está en', 'de américa', 'no no', 'en un',
           'perro está', 'esto es', 'muy bien', 'es la', 'el correo', 'la factura', 'bien el', 'tomar algo', 'es un',
           'quiere decir', 'con coches', 'es el', 'se vaya', 'un poco', 'perro de', 'duerme en', 'de compras',
           'compras para', 'en españa', 'le gusta', 'están las', 'una carta', 'te lo', 'se acabó', 'y no', 'no me',
           'lo siento', 'luis luis', 'que sí', 'correo luis', 'a ver', 'factura del', 'de la', 'oh es', 'es esto',
           'para ti', 'te quiere', 'américa américa', 'hace siete', 'siete años', 'español bueno']

bot = Bot(token=TOKEN, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
dp = Dispatcher(storage=MemoryStorage())

model = whisper.load_model("base")
user_state = {}

# --- Утилиты ---

def load_clips_from_excel(path=EXCEL_PATH):
    wb = load_workbook(path)
    ws = wb.active
    return [{"start": row[0], "end": row[1], "text": row[2]} for row in ws.iter_rows(min_row=2, values_only=True)]

clips = load_clips_from_excel()

def extract_clip(video_path: str, start: float, end: float, output_path: str):
    clip = VideoFileClip(video_path).subclip(start, end)
    clip.write_videofile(output_path, codec='libx264', audio_codec='aac', logger=None)

def generate_gap_text(sentence, keywords, bigrams):
    masked = sentence
    for bg in bigrams:
        masked = re.sub(rf"\b{re.escape(bg)}\b", "_____ _____", masked, flags=re.IGNORECASE)
    for kw in keywords:
        masked = re.sub(rf"\b{re.escape(kw)}\b", "_____", masked, flags=re.IGNORECASE)
    return masked

def transcribe(audio_path):
    result = model.transcribe(audio_path, language="es")
    return result["text"]

def compare(reference, recognized):
    matcher = difflib.SequenceMatcher(None, reference.lower().split(), recognized.lower().split())
    matched = sum(block.size for block in matcher.get_matching_blocks())
    total = max(len(reference.split()), len(recognized.split()))
    return matched / total if total > 0 else 0.0

# --- Кнопки ---

def navigation_keyboard():
    return InlineKeyboardMarkup(inline_keyboard=[
        [
            InlineKeyboardButton(text="🔁 Повторить", callback_data="repeat"),
            InlineKeyboardButton(text="⏭ Пропустить", callback_data="skip")
        ]
    ])

# --- Хендлеры ---

@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    user_state[message.chat.id] = 0
    await send_next_clip(message)

async def send_next_clip(message: types.Message):
    i = user_state.get(message.chat.id, 0)
    if i >= len(clips):
        await message.answer("🎉 Все фрагменты завершены!")
        return

    clip = clips[i]
    clip_path = os.path.join(FRAGMENTS_DIR, f"clip_{i}.mp4")

    if not os.path.exists(clip_path):
        extract_clip(VIDEO_PATH, clip['start'], clip['end'], clip_path)

    video = FSInputFile(clip_path)
    await bot.send_video(chat_id=message.chat.id, video=video)
    await message.answer(
        "📝 Вставь пропущенные слова:\n\n" + generate_gap_text(clip["text"], keywords, bigrams),
        reply_markup=navigation_keyboard()
    )

@dp.callback_query(lambda c: c.data in ["repeat", "skip"])
async def handle_navigation_buttons(callback: CallbackQuery):
    chat_id = callback.message.chat.id
    data = callback.data

    if chat_id not in user_state:
        await callback.answer("Пожалуйста, начни с /start")
        return

    if data == "repeat":
        await callback.message.answer("🔁 Повторим текущий фрагмент:")
    elif data == "skip":
        user_state[chat_id] += 1
        await callback.message.answer("⏭ Пропущено. Следующий фрагмент:")

    await send_next_clip(callback.message)
    await callback.answer()

@dp.message(F.voice | F.audio)
async def handle_audio(message: types.Message):
    if message.chat.id not in user_state:
        return

    i = user_state[message.chat.id]
    voice = message.voice or message.audio

    file = await bot.get_file(voice.file_id)
    file_path = os.path.join(USER_AUDIO_DIR, f"audio_{message.chat.id}_{i}.ogg")
    await bot.download_file(file.file_path, file_path)

    recognized = transcribe(file_path)
    reference = clips[i]["text"]
    score = compare(reference, recognized)

    await message.answer(f"🔊 Ты сказал: <i>{recognized}</i>\n📊 Произношение: <b>{score*100:.1f}%</b>")

    if score > 0.7:
        user_state[message.chat.id] += 1
        await send_next_clip(message)
    else:
        await message.answer("😕 Попробуй ещё раз — произношение пока не точное.")

@dp.message(F.text)
async def handle_text_response(message: types.Message):
    if message.chat.id not in user_state:
        return

    i = user_state[message.chat.id]
    correct = clips[i]["text"].lower().strip()
    answer = message.text.lower().strip()

    if answer == correct:
        await message.answer("✅ Верно! Теперь запиши аудио с этим предложением.")
    else:
        await message.answer("❌ Есть ошибки. Попробуй ещё раз.")

# --- Запуск бота ---

async def main():
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())

Received SIGINT signal
